# Training model
The goal of this notebook is to train a model and upload it to kaggle.

## Setup

### Imports

In [ ]:
import os
import json
from glob import glob
from functools import partial
from datetime import datetime
from itertools import pairwise
from os.path import join, realpath
from typing import Optional, Literal

import torch
import kagglehub
import numpy as np
import pandas as pd
import plotly.express as px
from torch import nn, Tensor
from pandas import DataFrame as DF
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader as DL
from rich.progress import Progress, Task, track
from torch.optim.lr_scheduler import ConstantLR, LRScheduler
metric_package = kagglehub.package_import('wasupandceacar/cmi-metric', bypass_confirmation=True)

### Config

In [103]:
NB_CROSS_VALIDATIONS = 5
TRAINING_EPOCHS = 60
STARTING_LR = 0.0005
BATCH_SIZE = 256
TARGET_NAMES = sorted([
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
])
TARGET_NAMES_NDARRAY = np.asarray(TARGET_NAMES)
MOCK_TRAINING_GAMMA = 1.01
MAX_LR_TO_MIN_DIV_FACTOR = 10

### Dataset Setup

In [37]:
dataset_path = kagglehub.dataset_download(
    handle="mauroabidalcarrer/prepocessed-cmi-2025",
)

In [38]:
class CMIDataset(TensorDataset):
    def __init__(
        self,
        parent_dir: str,
        split: Optional[Literal["train", "validation"]]=None,
        subset: Optional[int]=None,
        force_download=False
    ):
        dataset_path = kagglehub.dataset_download(
            handle="mauroabidalcarrer/prepocessed-cmi-2025",
            force_download=force_download
        )
        parent_dir = join(dataset_path, "preprocessed_dataset", parent_dir)
        split = "" if split is None else split + "_"
        x = np.load(join(parent_dir, f"{split}X.npy")).swapaxes(1, 2)
        y = np.load(join(parent_dir, f"{split}Y.npy"))
        if subset is not None:
            x = x[:subset]
            y = y[:subset]
        super().__init__(
            torch.from_numpy(x), 
            torch.from_numpy(y),
        )

In [39]:
meta_data_path = join(
    dataset_path,
    "preprocessed_dataset",
    "full_dataset_meta_data.json"
)
with open(meta_data_path, "r") as fp:
    meta_data = json.load(fp)
# Convert target names into a ndarray to index it batchwise.
# meta_data["target_names"] = np.asarray(meta_data["target_names"])
non_imu_feats_idx = [feat_idx for feat_idx, feat in enumerate(meta_data["feature_cols"]) if feat.startswith(("thm", "tof"))]
non_imu_feats = [feat for feat in meta_data["feature_cols"] if feat.startswith(("thm", "tof"))]
imu_feats_idx = [feat_idx for feat_idx, feat in enumerate(meta_data["feature_cols"]) if not feat.startswith(("thm", "tof"))]
imu_feats = [feat for feat in meta_data["feature_cols"] if not feat.startswith(("thm", "tof"))]

In [40]:
display(non_imu_feats)
display(imu_feats)

['thm_1',
 'thm_1_diff',
 'thm_2',
 'thm_2_diff',
 'thm_3',
 'thm_3_diff',
 'thm_4',
 'thm_4_diff',
 'thm_5',
 'thm_5_diff',
 'tof_1',
 'tof_1_diff',
 'tof_2',
 'tof_2_diff',
 'tof_3',
 'tof_3_diff',
 'tof_4',
 'tof_4_diff',
 'tof_5',
 'tof_5_diff']

['acc_x',
 'acc_x_diff',
 'acc_y',
 'acc_y_diff',
 'acc_z',
 'acc_z_diff',
 'angular_vel_x',
 'angular_vel_x_diff',
 'angular_vel_y',
 'angular_vel_y_diff',
 'angular_vel_z',
 'angular_vel_z_diff',
 'delta_rot_mag',
 'delta_rot_mag_diff',
 'euler_x',
 'euler_x_diff',
 'euler_y',
 'euler_y_diff',
 'euler_z',
 'euler_z_diff',
 'gravity_free_acc_x',
 'gravity_free_acc_x_diff',
 'gravity_free_acc_y',
 'gravity_free_acc_y_diff',
 'gravity_free_acc_z',
 'gravity_free_acc_z_diff',
 'linear_acc_mag',
 'linear_acc_mag_diff',
 'quat_rot_mag',
 'quat_rot_mag_diff',
 'raw_acc_mag',
 'raw_acc_mag_diff',
 'rot_w',
 'rot_w_diff',
 'rot_x',
 'rot_x_diff',
 'rot_y',
 'rot_y_diff',
 'rot_z',
 'rot_z_diff',
 'rotation_axis_x',
 'rotation_axis_x_diff',
 'rotation_axis_y',
 'rotation_axis_y_diff',
 'rotation_axis_z',
 'rotation_axis_z_diff']

In [41]:
display(non_imu_feats_idx)
display(imu_feats_idx)

[46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45]

### device setup

In [42]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Model definition

In [43]:
class ResidualBlock(nn.Module):
    def __init__(self, in_chns:int, out_chns:int):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Conv1d(in_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
            nn.ReLU(),
            nn.Conv1d(out_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
        )
        if in_chns == out_chns:
            self.skip_connection = nn.Identity() 
        else:
            # TODO: set bias to False ?
            self.skip_connection = nn.Sequential(
                nn.Conv1d(in_chns, out_chns, 1),
                nn.BatchNorm1d(out_chns)
            )

    def forward(self, x:Tensor) -> Tensor:
        activaition_maps = self.skip_connection(x) + self.blocks(x)
        return nn.functional.relu(activaition_maps)

class Resnet(nn.Module):
    def __init__(
            self,
            in_channels:int,
            depth:int,
            # n_res_block_per_depth:int,
            mlp_width:int,
            n_class:int,
        ):
        super().__init__()
        chs_per_depth = [in_channels * 2 ** i for i in range(depth)]
        blocks_chns_it = pairwise(chs_per_depth)
        self.res_blocks = [ResidualBlock(in_chns, out_chns) for in_chns, out_chns in blocks_chns_it]
        self.res_blocks = nn.ModuleList(self.res_blocks)
        self.mlp_head = nn.Sequential(
            nn.LazyLinear(mlp_width),
            nn.ReLU(),
            nn.Linear(mlp_width, n_class),
            nn.Softmax(dim=1),
        )
        
        
    def forward(self, x:Tensor) -> Tensor:
        activation_maps = x
        for res_block in self.res_blocks:
            activation_maps = nn.functional.max_pool1d(res_block(activation_maps), 2)
        out = activation_maps.view(activation_maps.shape[0], -1)
        out = self.mlp_head(out)
        return out

## Training functions 

### Evaluation 

In [44]:
def evaluate_model(
        model: torch.nn.Module,
        critirion:callable,
        validation_loader:DL,
    ) -> dict:
    model = model.eval()
    model_device = next(model.parameters()).device
    competition_metric = metric_package.Metric()

    total_accuracy = 0
    total_test_loss = 0
    with torch.inference_mode():
        for x, y in validation_loader:
            # Additionally, clone x because we will write on it, in theory it's not necessary but we never know/
            x = x.to(model_device).clone()
            y = y.to(model_device)

            half = BATCH_SIZE // 2

            x[:half, non_imu_feats_idx, :] = 0.0

            y_pred = model(x)
            total_test_loss += critirion(y_pred, y).item()
            batch_accuracy = compute_accuracy(y_pred, y)
            total_accuracy += batch_accuracy
            competition_metric.add(get_str_target(y), get_str_target(y_pred))

    return {
        "validation_loss": total_test_loss / len(validation_loader),
        "validation_accuracy": total_accuracy / len(validation_loader),
        "competition_score": competition_metric.score(),
    }

def get_str_target(y:Tensor) -> str:
    y_idx = y.max(dim=1)[1].tolist()
    return TARGET_NAMES_NDARRAY[y_idx]

def compute_accuracy(y_pred:Tensor, y:Tensor) -> float:
    return (
            (y.max(dim=1)[1] == y_pred.max(dim=1)[1])
            .sum()
            .item()
    ) / y.shape[0]

### Checkpoints

In [45]:
def mk_checkpoint(
        epoch:int,
        model: torch.nn.Module,
        scheduler: LRScheduler,
        optimizer: torch.optim.Optimizer
    ) -> dict:
    if torch.cuda.is_available():
        model_state_dict = {k: v.detach().cpu() for k, v in model.state_dict().items()}
    else:
        model_state_dict = model.state_dict()

    return {
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict(),
    }

def save_checkpoint(checkpoint:dict, parent_dir:str, is_best_checkpoint=False):
    # Create model name
    checkpoint_filename = f"epoch_{checkpoint['epoch']}_{mk_date_now_str()}.pth"
    # Save model
    os.makedirs(parent_dir, exist_ok=True)
    checkpoint_path = os.path.join(parent_dir, checkpoint_filename)
    torch.save(checkpoint, checkpoint_path)
    mk_symlink(checkpoint_path, os.path.join(parent_dir, "latest_checkpoint.pth"))
    if is_best_checkpoint:
        mk_symlink(checkpoint_path, os.path.join(parent_dir, "best_checkpoint.pth"))

def mk_date_now_str() -> str:
    return datetime.now().strftime("%d-%m-%Y_%H-%M")

def mk_symlink(dest:str, symlink_path:str):
    if os.path.islink(symlink_path) or os.path.exists(symlink_path):
        os.remove(symlink_path)
    os.symlink(dest, symlink_path)

### Training loop

In [53]:
def fit(epochs:int,
        model: nn.Module,
        scheduler: LRScheduler,
        optimizer: torch.optim.Optimizer,
        train_loader: DL,
        criterion: callable=nn.L1Loss(),
        evaluation_func: callable=None,
        validation_loader: DL=None,
        save_checkpoints=True,
    ) -> tuple[DF, str]:
    """
    Returns:
        (training_metrics, path_to_checkpoints)
    """
    # Setup
    metrics: list[dict] = []
    checkpoints_dir = os.path.join("checkpoints", mk_date_now_str())
    step = 0
    model_device = next(model.parameters()).device
    last_epoch_metric = {}
    # Training loop
    with Progress() as progress:
        task: Task = progress.add_task(
            "training...",
            total=len(train_loader),
        )
        for epoch in range(epochs):
            progress.update(
                task,
                description=f"epoch: {epoch}",
                completed=0,
            )
            total_epoch_loss = 0
            total_accuracy = 0
            for batch_idx, (x, y) in enumerate(train_loader):
                # forward
                x = x.to(model_device)
                y = y.to(model_device)
                model.train()
                optimizer.zero_grad()
                y_pred: Tensor = model(x)
                loss_value = criterion(y_pred, y)
                # Verify loss value
                if torch.isnan(loss_value).any().item():
                    progress.print("Warning: Got NaN loss, stopped training.")
                    return DF.from_records(metrics), checkpoints_dir
                if torch.isinf(loss_value).any().item():
                    progress.print("Warning: Got infinite loss, stopped training.")
                    return DF.from_records(metrics), checkpoints_dir
                # TODO: Use gradient clipping?
                loss_value.backward()
                optimizer.step()
                if step > 0: # If it's not the first training step, idk why it throws an error otherwise
                    scheduler.step()
                # metrics
                total_epoch_loss += loss_value.item()
                batch_accuracy = compute_accuracy(y_pred, y)
                total_accuracy += batch_accuracy
                metrics.append({
                    "step": step,
                    "epoch": epoch,
                    "batch_train_loss": loss_value.item(),
                    "lr": optimizer.state_dict()["param_groups"][-1]["lr"],
                    "batch_accuracy": batch_accuracy,
                })
                step += 1
                if "validation_accuracy" in last_epoch_metric:
                    last_validation_acc = "%.2f" % last_epoch_metric["validation_accuracy"]
                    val_acc_str = "val. acc: " + last_validation_acc
                else:
                    val_acc_str = ""
                progress.update(
                    task,
                    advance=1,
                    description=f"epoch: {epoch}, batch_loss: {(total_epoch_loss / (batch_idx+1)):.2f}, {val_acc_str}"
                )
            # Post epoch evalution
            metrics[-1]["train_epoch_loss"] = total_epoch_loss / len(train_loader)
            metrics[-1]["train_epoch_accuracy"] = total_accuracy / len(train_loader)
            if evaluation_func:
                progress.update(
                    task,
                    completed=0,
                    description=f"epoch: {epoch}, evaluating..."
                )
                eval_metrics = evaluation_func(model, criterion, validation_loader)
                metrics[-1].update(eval_metrics)
            last_epoch_metric = metrics[-1]
            # Save checkpoint
            if save_checkpoints:
                checkpoint = mk_checkpoint(epoch, model, scheduler, optimizer)
                metrics_df = DF.from_records(metrics)
                best_model_metric = "validation_loss" if "validation_loss" in metrics_df.columns else "train_epoch_loss"
                is_best_checkpoint = (
                    DF.from_records(metrics)
                    .eval(f"min_{best_model_metric} = {best_model_metric}.min()")
                    .eval(f"is_best_{best_model_metric} = {best_model_metric} == min_{best_model_metric}")
                    .dropna(subset=f"is_best_{best_model_metric}")
                    .iloc[-1]
                    .loc[f"is_best_{best_model_metric}"]
                )
                save_checkpoint(checkpoint, checkpoints_dir, is_best_checkpoint)

    return DF.from_records(metrics), checkpoints_dir

### Create model function

In [47]:
def mk_model() -> nn.Module:
    nb_in_chans = len(meta_data["feature_cols"])
    return (
        Resnet(
            in_channels=nb_in_chans,
            depth=4,
            mlp_width=256,
            n_class=18
        )
        .to(device)
    )

print("input channels:", len(meta_data["feature_cols"]))

input channels: 66


### Create model and train it

In [ ]:
def mk_model_and_fit(
        train_loader:DL,
        starting_lr:float,
        mk_scheduler:callable,
        validation_loader:Optional[DL]=None,
        save_checkpoints=False,
    ) -> tuple[nn.Module, DF, list[str]]:
    model = mk_model()
    optimizer = torch.optim.AdamW(model.parameters(), STARTING_LR)
    lr_scheduler = mk_scheduler(optimizer)
    training_metrics, model_checkpoints = fit(
        epochs=TRAINING_EPOCHS,
        model=model,
        scheduler=lr_scheduler,
        optimizer=optimizer,
        train_loader=train_loader,
        criterion=nn.CrossEntropyLoss(),
        evaluation_func=evaluate_model if validation_loader else None,
        validation_loader=validation_loader,
        save_checkpoints=save_checkpoints,
    )

    return model, training_metrics, model_checkpoints

## Search max learning rate

In [96]:
def post_process_mock_training_metrics(training_metrics:DF) -> DF:
    training_metrics = (
        training_metrics
        .query("batch_train_loss.notna()")
        .set_index("lr", drop=False)
        .sort_index()
    )
    training_metrics["ewm_batch_train_loss"] = (
        training_metrics
        .ewm(com=30, ignore_na=False)
        ["batch_train_loss"]
        .mean()
    )
    training_metrics["ewm_batch_train_loss_diff"] = training_metrics["ewm_batch_train_loss"].diff()
    return training_metrics

In [ ]:
def plt_lr_search_training_metrics(training_metrics:DF):
    (    
        px.line(
            (
                training_metrics
                .reset_index(drop=True)
                .melt(
                    id_vars="lr",
                    value_vars=[
                        "batch_train_loss",
                        "ewm_batch_train_loss",
                        "ewm_batch_train_loss_diff",
                    ],
                )
            ),
            x="lr",
            facet_row="variable",
            y="value",
            log_x=True,
            height=750,
        )
        .update_yaxes(matches=None)
        .show()
    )

In [100]:
_, mock_training_metrics, _ = mk_model_and_fit(
    full_dataset_loader,
    partial(torch.optim.lr_scheduler.ExponentialLR, gamma=MOCK_TRAINING_GAMMA),
)
mock_training_metrics = post_process_mock_training_metrics(mock_training_metrics)
plt_lr_search_training_metrics(mock_training_metrics)
max_lr = mock_training_metrics["ewm_batch_train_loss"].idxmin()
print("Maximum learning rate:", max_lr)

Output()

Warning: Got NaN loss, stopped training.

Maximum learning rate: 0.002361045271265381


## Cross validation training

In [101]:
def plt_cross_validation_training_metrics(training_metrics:DF):
    (
        px.line(
            (
                training_metrics
                .query("validation_loss.notna()")
                .melt(
                    id_vars="epoch",
                    value_vars=['train_epoch_accuracy', 'validation_accuracy', "competition_score"]
                )
            ),
            x="epoch",
            y="value",
            color="variable",
            facet_row="variable",
            title="epoch metrics",
            render_mode="line+marker",
        ).update_yaxes(matches=None)
        .show()
    )

In [108]:
fold_patterns = join(dataset_path, "preprocessed_dataset", "fold*")
fold_pths = glob(fold_patterns)[:NB_CROSS_VALIDATIONS]
all_training_metrics = {}

for fold_idx, fold_pth in enumerate(fold_pths):
    print("training:", fold_idx + 1)
    train_dataset = CMIDataset(fold_pth, "train")
    train_data_loader = DL(train_dataset, BATCH_SIZE, shuffle=True)
    validation_dataset = CMIDataset(fold_pth, "validation")
    validation_data_loader = DL(validation_dataset, BATCH_SIZE, shuffle=False)
    _, training_metrics, _ = mk_model_and_fit(
        train_data_loader,
        validation_loader=validation_data_loader, 
        mk_scheduler=partial(
            torch.optim.lr_scheduler.OneCycleLR,
            max_lr=max_lr / 2,
            div_factor=MAX_LR_TO_MIN_DIV_FACTOR,
            epochs=TRAINING_EPOCHS,
            steps_per_epoch=len(train_data_loader),
        )
    )
    all_training_metrics["fold_" + str(fold_idx)] = training_metrics
    plt_cross_validation_training_metrics(training_metrics)
    print("=========================")

if NB_CROSS_VALIDATIONS:
    all_training_metrics = pd.concat(all_training_metrics)

training: 1


Output()

training: 2


Output()

training: 3


Output()

training: 4


Output()

training: 5


Output()

In [106]:
if NB_CROSS_VALIDATIONS:
    display(
        all_training_metrics
        .loc[:,
            [
                "train_epoch_loss",
                "train_epoch_accuracy",
                "validation_loss",
                "validation_accuracy",
                "competition_score",
            ],
        ]
        .agg(["mean", "std"])
    )

,train_epoch_loss,train_epoch_accuracy,validation_loss,validation_accuracy,competition_score
mean,2.537597,0.443416,2.651108,0.329338,0.563472
std,0.071056,0.071189,0.055084,0.055335,0.044661


## Full dataset training

In [107]:
def plt_full_dataset_training_metrics(training_metrics:DF):
    (
        px.scatter(
            (
                training_metrics
                .melt(
                    id_vars="step",
                    value_vars=[
                        "batch_train_loss",
                        "batch_accuracy",
                    ]
                )
            ),
            x="step",
            y="value",
            facet_row="variable",
            trendline="ewm",
            trendline_options={"com": 30},
            trendline_color_override="red",
            title="batch metrics",
        )
        .update_yaxes(matches=None)
        .show()
    )

In [110]:
full_dataset = CMIDataset("full_dataset")
full_dataset_loader = DL(full_dataset, BATCH_SIZE, shuffle=True)
model, full_train_metrics, checkpoints = mk_model_and_fit(
    full_dataset_loader,
    mk_scheduler=partial(
        torch.optim.lr_scheduler.OneCycleLR,
        max_lr=max_lr / 2,
        div_factor=MAX_LR_TO_MIN_DIV_FACTOR,
        epochs=TRAINING_EPOCHS,
        steps_per_epoch=len(full_dataset_loader),
    )
)
plt_full_dataset_training_metrics(full_train_metrics)

Output()

## Model upload

In [ ]:
handle = f"{kagglehub.whoami()['username']}/cmi-resnet/pyTorch/{mk_date_now_str().replace('_', '-')}"
model_path = realpath(join(checkpoints))
display(model_path)
kagglehub.model_upload(handle, model_path)

UnauthenticatedError: User is not authenticated